In [1]:
import scipy.io
import cv2
import pandas as pd
import os
import random 
import numpy as np
import json
import math

In [93]:
def parseVideoName(video_name):
    video_name = video_name.replace('.avi', '')
    first_part,second_part = video_name.split(',')
    video_type, num_targets = first_part.rsplit('_', 1)
    videos_info = second_part.split('_')
    
    parsed_data = {
        'video_name': video_name,
        'target_name': second_part,
        'video_type': video_type,
        'num_targets': int(num_targets),
        'first_video': int(videos_info[0]),
        'second_video': int(videos_info[1]),
        'first_start_frame': int(videos_info[2]),
        'second_start_frame': int(videos_info[3]),
        'first_rotation': float(videos_info[4]),
        'second_rotation': float(videos_info[5])
    }
    
    return parsed_data
    
def generateRandomColor():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def processTrialsVideos(file_path):
    df = pd.read_csv(file_path)
    parsed_rows = []
    
    for index, row in df.iterrows():
        video_name = row['videoName']
        parsed_data = parseVideoName(video_name)
        parsed_data['firstTargets'] = row['firstTargets']
        parsed_data['secondTargets'] = row['secondTargets']
        parsed_rows.append(parsed_data)
        # parsed_rows.append(1)
    return pd.DataFrame(parsed_rows)
    
def loadMatFile():
    root_folder = 'GazeMat'
    test_subs = ['sub11', 'sub21', 'sub31', 'sub41', 'sub51']

    data = {}
    
    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            mat_file_path = os.path.join(root_folder, sub_dir, '40.mat')
            if os.path.exists(mat_file_path):
                mat_data = scipy.io.loadmat(mat_file_path)
                data[sub_dir] = mat_data["extractedData"]
            else:
                print(f"{mat_file_path} does not exist.")
    print("Data loaded successfully.")
    return data


def loadShapeFishPoints(videoName,frame):
    print(frame)
    json_filename = f'img{frame:03d}.json'
    json_path = os.path.join('./jsons', str(videoName), json_filename)

    if not os.path.exists(json_path):
        raise FileNotFoundError(f"JSON file not found: {json_path}")

    with open(json_path, 'r') as json_file:
        data = json.load(json_file)
    
    return data.get('shapes', [])

def getTargetsFromJson(shapes,targets):
    if not isinstance(targets, (list, tuple)):
        targets = [targets]
    
    target_digits = set()
    for target in targets:
        if isinstance(target, int):
            target_digits.update(str(target))
        else:
            raise ValueError("Targets should be integers or lists/tuples of integers")

    filtered_shapes = [shape for shape in shapes if shape['label'] in target_digits]
    
    return filtered_shapes

def getTotalShapeFirst(shapeInfo,baseFrame):
    firstFrame = shapeInfo['firstStart'] + baseFrame
    firstTargets = shapeInfo['firstTargets']
    shapes_video1 = loadShapeFishPoints(shapeInfo['first'], firstFrame)
    return getTargetsFromJson(shapes_video1,firstTargets)
    
def getTotalShapeSecond(shapeInfo,baseFrame):
    secondFrame = shapeInfo['secondStart'] + baseFrame
    secondTargets = shapeInfo['secondTargets']
    shapes_video2 = loadShapeFishPoints(shapeInfo['second'], secondFrame)
    merged_shapes = getTargetsFromJson(shapes_video2,secondTargets)
    return merged_shapes

def generateSubjectName(subIndex):
    number_part = subIndex[3:]
    modified_number = number_part[:-1] if len(number_part) > 1 else '0'
    return f'sub{modified_number}'

def getVideoInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # num_frames = int(mainVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height 

def createOutputVideo(video,name):
    output_gaze_videos = 'test'
    os.makedirs(output_gaze_videos, exist_ok=True)
    fps, width, height = getVideoInfo(video)
    
    output_gaze_videos_path = os.path.join(output_gaze_videos, name + '.avi')
    output_video = cv2.VideoWriter(output_gaze_videos_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    return output_video

def getSubjectColor(subject):
    if subject not in subject_colors:
        subject_colors[subject] = generateRandomColor()
    return subject_colors[subIndex]

def mapPointsOnFrame(frame):
    return round(frame * (7798 / 110))

def openVideo(video):
    trialVideo = cv2.VideoCapture(os.path.join('./videos', video['video_type'], video['target_name'] + '.avi'))
    
    if not trialVideo.isOpened():
        raise ValueError(f"Unable to open video file: {os.path.join('./videos', video['video_type'], video['target_name'] + '.avi')}")
    
    return trialVideo

def generateXPoints(x):
    scale_x = 660 / 1920
    return int(x * scale_x)

def generateYPoints(y):
    scale_y = 660 / 1080
    return int(y * scale_y)

def rotate_frame(frame, angle):
    (h, w) = frame.shape[:2]
    center = (330, 330)
    
    # Create the rotation matrix
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # Perform the rotation
    rotated_frame = cv2.warpAffine(frame, M, (w, h))
    return rotated_frame

In [94]:
subjects = loadMatFile()
subject_colors = {}

Data loaded successfully.


In [95]:
videoData = processTrialsVideos('./sub21.csv').iloc[1:2]
videoData

,video_name,target_name,video_type,num_targets,first_video,second_video,first_start_frame,second_start_frame,first_rotation,second_rotation,firstTargets,secondTargets
1,"NO_2,1_10_119_344_45_90",1_10_119_344_45_90,NO,2,1,10,119,344,45.0,90.0,3,4


In [99]:
def draw_shapes(frame, shapes):
    for shape in shapes:
        points = shape['points']
        # Check if points are empty
        if not points:
            continue
        
        # Calibrate points
        calibrated_points = [(generateXPoints(x), generateYPoints(y)) for x, y in points]
        # Convert to NumPy array and reshape
        np_points = np.array(calibrated_points, dtype=np.int32).reshape((-1, 1, 2))
        
        # Debug print
        print(f"Drawing shape with points: {np_points}")

        # Ensure np_points is valid
        if np_points.size == 0:
            continue
        
        # Draw the shape
        cv2.polylines(frame, [np_points], isClosed=True, color=(0, 0, 225), thickness=2)
        cv2.fillPoly(frame, [np_points], color=(0, 0, 225))

In [100]:
videoList = []
trialNumber  = 1

for index, video in videoData.iterrows():
    try:
        
        open_video = openVideo(video)
        output_video = createOutputVideo(open_video,video.video_name)
        
        frame_idx = 0

        while True:
            ret, frame = open_video.read()
            if not ret:
                break  

            shapeInfo = {
                'first': video.first_video,
                'second': video.second_video,
                'firstTargets': video.firstTargets,
                'secondTargets': video.secondTargets,
                'firstStart': video.first_start_frame,
                'secondStart': video.second_start_frame
            }

            shapeFirst = getTotalShapeFirst(shapeInfo, frame_idx)
            shapeSecond = getTotalShapeSecond(shapeInfo, frame_idx)
            
            # Rotate the frame for the first and second videos
            rotated_frame_first = rotate_frame(frame, video.first_rotation)
            rotated_frame_second = rotate_frame(frame, video.second_rotation)

            # Draw shapes for the first video
            print("Processing shapes from first video")
            draw_shapes(rotated_frame_first, shapeFirst)

            # Draw shapes for the second video
            print("Processing shapes from second video")
            draw_shapes(rotated_frame_second, shapeSecond)

            # Optionally, choose which rotated frame to save
            # For example, save the first rotated frame
            resized_frame_first = cv2.resize(rotated_frame_first, (660, 660))
            output_video.write(resized_frame_first)
            frame_idx += 1
        
        trialNumber += 1
        
    except Exception as e:
        print(f"An error occurred while processing video at index {index}: {e}")

    finally:
        # Release everything
        if 'open_video' in locals() and open_video.isOpened():
            open_video.release()
        if 'output_video' in locals():
            output_video.release()
        cv2.destroyAllWindows()
    

119
344
Processing shapes from first video
Drawing shape with points: [[[244 265]]

 [[239 255]]

 [[232 271]]

 [[227 281]]

 [[222 293]]

 [[214 315]]

 [[223 297]]

 [[229 286]]

 [[234 275]]]
Processing shapes from second video
Drawing shape with points: [[[ 80 378]]

 [[ 82 387]]

 [[118 344]]]
120
345
Processing shapes from first video
Drawing shape with points: [[[244 265]]

 [[239 255]]

 [[232 271]]

 [[227 281]]

 [[222 293]]

 [[214 315]]

 [[223 297]]

 [[229 286]]

 [[234 275]]]
Processing shapes from second video
Drawing shape with points: [[[ 80 378]]

 [[ 82 387]]

 [[118 344]]]
121
346
Processing shapes from first video
Drawing shape with points: [[[244 265]]

 [[239 255]]

 [[232 271]]

 [[227 281]]

 [[222 293]]

 [[214 315]]

 [[223 297]]

 [[229 286]]

 [[234 275]]]
Processing shapes from second video
Drawing shape with points: [[[ 80 378]]

 [[ 82 387]]

 [[118 344]]]
122
347
Processing shapes from first video
Drawing shape with points: [[[243 262]]

 [[237 257]]
